In [1]:
import module_luis.get_data as get_data
import module_luis.luis_function as lf
import module_luis.luis_prediction as luis_predict

runtime_key: ef3b354d5d1b42459bf2e2591519cba2
runtime_endpoint: https://westeurope.api.cognitive.microsoft.com/


<div class="alert alert-block alert-info">
<h2><center><strong>Creation des données pour Luis </strong></center></h2>
        
</div>

In [2]:
%load_ext pycodestyle_magic

In [3]:
%flake8_on

<div class="alert alert-block alert-success">  
<strong>1. Chargement du jeu de données<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

*a) Initialisation des paramètres*

In [2]:
data_path = "data/frames.json"

*b) Chargement du fichier*

In [3]:
data_list = get_data.load_data(data_path)

On le transforme pour qu'il soit utilisable dans luis :

In [4]:
list_final = get_data.transform_data(data_list)
list_train = list_final[:-99]
list_test = list_final[-99:]

### Creation de l'application luis, des intentions et des utterances :

In [5]:
print("Creating application...")
app_id, app_version = lf.create_app()
print(app_id, app_version)

Creating application...
Created LUIS app Fly Bot
    with ID 482c7139-dbf6-4560-be9b-821f34dc99b9
482c7139-dbf6-4560-be9b-821f34dc99b9 0.1


In [6]:
print("Adding entities to application...")
lf.add_entities(app_id, app_version)

Adding entities to application...
destinationEntityId abdfcf8f-3f74-4622-85c7-46d977c951f9 added.
classEntityId 3af1b04a-4567-46de-bdae-3c3bf2a8f987 added.
flightEntityId 66298b19-4351-4013-9470-57d23d3a68c2 added.
flightEntityId 781d33a4-0a9a-47ae-97c8-928195b2e533 added.
flightEntityId c2320e13-3bd7-42f7-af2a-42b3916f1e79 added.
flightEntityId [<azure.cognitiveservices.language.luis.authoring.models.prebuilt_entity_extractor_py3.PrebuiltEntityExtractor object at 0x00000248002CCDC0>] added.


In [7]:
print("Adding intents to application...")
lf.add_intents(app_id, app_version)

Adding intents to application...
Intent BookFlight a2da9561-9d2f-44e9-8305-44fa62ded6c0 added.
Intent Cancel 7208b408-49f0-4c71-8871-05d3cce6cdbc added.
Intent Greeting f51c9d09-2df7-4835-b0d7-2926f6ab1406 added.
Intent Help ecddd9f0-0ba7-4892-a6e4-f07fe3c5bb8c added.
Intent Closing 62f82277-edca-4945-894a-f8c306d6dc94 added.


on ajoute les entités correspondant à Greeting, Cancel, Help, None...

In [8]:
intentions_path = "data/data_intentions.txt"
list_utterances_default = get_data.create_data(intentions_path)
lf.add_utterances(app_id, app_version, list_utterances_default)
print("{} example utterance(s) added.".format(len(list_utterances_default)))

45 example utterance(s) added.


On les ajoute 100 par 100 sinon Luis refuse autant de requetes en gratuit : 

In [9]:
print("Adding intents to utterances...")
# on garde les 229 premieres utterances pour evaluer luis
for i in range(229, len(list_train), 100):
    j = (i + 100)
    if j > len(list_final):
        j = len(list_final)
    lf.add_utterances(app_id, app_version, list_train[i:j])
print("{} example utterance(s) added.".format(len(list_train)))

Adding intents to utterances...
5130 example utterance(s) added.


In [10]:
print("Train application...")
lf.train_app(app_id, app_version)

Train application...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
Waiting 10 seconds for training to complete...
trained


In [8]:
print("Evaluate application...")
test_path = "data/evaluate.json"
lf.evaluate_app(test_path, list_test)

Evaluate application...


In [11]:
print("Publish application...")
lf.publish_app(app_id, app_version)

Publish application...
Application published. Endpoint URL: https://westeurope.api.cognitive.microsoft.com/luis/v2.0/apps/482c7139-dbf6-4560-be9b-821f34dc99b9


<div class="alert alert-block alert-success">  
<strong>2. Evaluation des predictions<a id='head-1-2'></a> 📕 📗 📘 📙</strong>
        
</div>

In [11]:
# <OtherVariables>
# Use public app ID or replace with your own trained and published app's ID
# to query your own app
# public appID = 'df67dcdb-c37d-46af-88e1-8b97951ca1c2'
luisAppID = '5705e735-228d-430c-94fd-cab67b7f69ae'
print("luisAppID: {}".format(luisAppID))

# `production` or `staging`
luisSlotName = 'production'
print("luisSlotName: {}".format(luisSlotName))
# </OtherVariables>


luisAppID: 5705e735-228d-430c-94fd-cab67b7f69ae
luisSlotName: production


In [13]:
luis_predict.predict(luisAppID, luisSlotName)

Top intent: BookFlight
Sentiment: None
Intents: 
	"BookFlight"
Entities: {'or_city': ['paris'], 'dst_city': ['berlin'], 'budget': ['2020'], 'str_date': ['september'], 'end_date': ['2nd october']}
